# Imports sous Notebook Python
Dans ce TP, vous allez essentiellement programmer des classes qui vous permettront de construire des réseaux de neurones. Ces classes seront enregistrées dans un fichier `Neural.py` qui contient déjà la classe `Generic`. Cependant, le comportement par défaut d'un Notebook quand on demande d'importer un fichier est de ne pas le relire !!! Ainsi vos modifications dans le fichier `Neural.py` ne seront pas prises en compte. Pour que ce soit le cas, il faut lancer les commandes suivantes :

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import Neural as Neur

In [3]:
import Neural as Neur
L=Neur.Generic()
print(L.backward(1))
# Vous devez trouver
# (None)

None


Copiez-collez la classe `Generic` en une classe `Arctan` et essayez de voir si python recharge bien le fichier Neural quand vous lancez la commande `import`



In [9]:
import Neural as Neur
L=Neur.Arctan()
print(L.backward(1))
# Vous devez trouver
# (None, None)

None


# Définition d'une couche
Le fichier `Neural.py` sera une librairie qui nous permettra de construire nos réseaux de neurones, nous allons au fur et à mesure construire nos différent types de couche.
Toute nos couches auront par défaut au moins les mêmes fonctions et variables que la couche `Generic`, et elles peuvent en avoir plus. Mathématiquement, on rappelle que une couche est une fonction $\mathcal F$ qui prend des données $x$ et qui rend un vecteur $y$. La couche a en interne des `paramètres` qui ont vocation à êtres `appris`. Les paramètres sont notés $\theta$. On a ainsi
$$ y=\mathcal F(x,\theta)$$

Les fonctions sont :

* `forward` : Applique la fonction $\mathcal F$ aux données $x$, récupère les paramètres internes $\theta$ et rend $y$.
* `backward` : Applique la rétro-propagation du gradient à un gradient de sortie `hat_y` et calcule 
  * le gradient des paramètres : `hat_theta` qui est le gradient par rapport aux paramètres internes
  * le gradient d'entrée `hat_x` qui est le gradient par rapport à la variable $x$

Les variables internes sont :
* `self.list_params` : une liste des paramètres $\theta$.
* `self.save` : Permet de sauver des informations dans la méthode `forward` si la méthode `backward` en a besoin.

On doit se souvenir d'un truc concernant les tailles de vecteur.
* Le vecteur $x$ et `hat_x` sont de même taille (`hat_x` est le gradient par rapport à $x$).
* Le vecteur $y$ et `hat_y` sont de même taille (`hat_y` est le gradient par rapport à $y$).
* Le vecteur $\theta$ et `hat_theta` sont de même taille (`hat_theta` est le gradient par rapport à $\theta$).

Attention en fonction de ce que vous sauvez dans la variable `self.save`, votre réseau de neurones très gourmand en mémoire. D'autres choix peuvent être faits, mais nous nous fixerons sur ces choix là pendant les TPs.



# Implémentation de la couche Arctan
Passons maintenant à notre première couche, vous avez normalement copié-collé la classe `Generic` en une classe `Arctan`. Nous allons remplir cette classe.

La couche `Arctan` est une couche qui prend comme vecteur d'entrée $x$ de taille $x$ et qui rend un vecteur $y$ de taille $p$ tel que
$$y[i]=\phi(x[i]) \quad \forall 1\le i\le p$$

Où $\phi$ est la fonction arctangente. Cette couche n'a pas de paramètres locaux (`self.list_params=[]`). Le backward de cette couche est :
$$ \hat x[i]=\phi'(x[i])\hat y[i] \quad \forall 1\le i\le p$$
où $\hat x$ et $\hat y$ sont respectivement le gradient d'entrée et le gradient de sortie. On rappelle que la variable `x` a été sauvegardée dans la variable `self.save`.

Implémentez cette couche et testez le code suivant, pour le gradient local, ou pour `get_params` on rendra la valeur `None`.

In [10]:
import Neural as Neur
L=Neur.Arctan()
np.random.seed(10)
X=np.random.randn(4,2)
grad_sortie=np.random.randn(4,2)
print('nb_params=',len(L.list_params))
print('structure des params :', [p.data.shape for p in L.list_params])
print('forward=',L.forward(X))
print('backward=',L.backward(grad_sortie))
# Vous devez trouver
#nb_params= 0
#structure des params : []
#forward= [[ 0.92666583  0.62090688]
# [-0.99647548 -0.00838365]
# [ 0.55596017 -0.6240794 ]
# [ 0.25952369  0.10812518]]
#backward=  [[ 0.00154751, -0.11550505],
#       [ 0.12780186,  1.20295282],
#       [-0.69626624,  0.67715401],
#       [ 0.21357394,  0.43995373]]


nb_params= 0
structure des params : []
forward= [[ 0.92666583  0.62090688]
 [-0.99647548 -0.00838365]
 [ 0.55596017 -0.6240794 ]
 [ 0.25952369  0.10812518]]
backward= [[ 0.00154751 -0.11550505]
 [ 0.12780186  1.20295282]
 [-0.69626624  0.67715401]
 [ 0.21357394  0.43995373]]


# D'autres couches
Sur le modèle de la couche Arctan, on peut construire un certain nombre de couches en modifiant la fonction $\Phi$ (et évidemment la fonction $\Phi'$). Ce sont toutes des couches simples sans paramètres et qui ne font qu'appliquer une non-linéarité aux données. Voici un tableau de quelques couches utilisées, de leur $\Phi$ et $\Phi'$ correspondant :


| Nom            |     $\Phi(X)$        |        $\Phi'(X)$                 |
| :------------  | :---------------:    | ----------------------:           |
| Sigmoïde       | $$\frac 1 {1+e^{-X}}$$ | $$\frac {e^{-X}} {(1+e^{-X})^2}$$ |
| RELU           |   $$\max(X,0)$$        |   $$\max(\frac{X}{|X|},0)$$          |
| ABS            |     $$|X|$$            |        $$\frac{X}{|X|}$$              |

# La structure des données
Avant de continuer, il nous faut dire quelle est la structure des données.
On suppose que l'on a $n$ données différentes dans $\mathbb{R}^p$. Ces données sont stockées dans une grande matrice de taille $(p,n)$ dont la $j$-ème colonne est un vecteur de taille $\mathbb{R}^p$ qui représente la $j$-eme donnée.
On note cette matrice $X_j[i]$ avec $1\le i \le p$ et $1\le j \le n$ tel que le vecteur $X_j$ est la $j$-eme donnée d'entrée.
Ainsi l'exemple suivant représente 4 données dans $\mathbb{R}^3$

In [15]:
X =np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
print(X)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


La première opération que l'on veut faire est, étant donné une matrice $A$ de taille $(q,p)$, de trouver la matrice $Y$ de taille $(q,n)$ telle que pour chaque donnée $j$, on ait $Y_j=AX_j$. Dans l'exemple suivant $q=2$.

In [16]:
A=np.array([[1,2,3],[4,5,6]])
print('A=',A)
Y = np.zeros((2,4))
for i in range(4):
    Y[:,i] = A@X[:,i]
print ('Y=',Y)
# Vous devez trouver
#A= [[1 2 3]
# [4 5 6]]
#Y= [[ 38  44  50  56]
# [ 83  98 113 128]]

A= [[1 2 3]
 [4 5 6]]
Y= [[ 38.  44.  50.  56.]
 [ 83.  98. 113. 128.]]


Maintenant on veut ajouter à $Y$ un vecteur $b \in \mathbb{R}^q$ tel que pour chaque donnée $j$ on ait $Z_j=Y_j+b$. Les exemples ci-dessous doivent vous aider à comprendre que la commande `Y+b` ne fonctionne pas. Par contre si on considère que  $b$ est un vecteur  $b\in \mathbb{R}^{q\times 1}$, alors `Y+b` la commande fonctionne.

In [18]:
b=np.array([1,2])
print(b.shape)
print(Y)
#print(Y+b) ## Cela pose un problème, commentez cette ligne
c=b.reshape((2,1))
print(c)    
print(Y+c) ## C'est exactement ce que l'on veut !!


(2,)
[[ 38.  44.  50.  56.]
 [ 83.  98. 113. 128.]]
[[1]
 [2]]
[[ 39.  45.  51.  57.]
 [ 85. 100. 115. 130.]]


 Vous devez vous servir de ce que vous avez appris pour calculer le vecteur $Z$ tel que $Z_j=AX_j+b$ dans l'exemple suivant :

In [19]:
X =np.array([[1,1,3,4],[5,6,7,8],[9,10,11,12]])
A=np.array([[1,5,3],[4,5,6]])
b=np.array([1,3]).reshape((2,1)) # Modifiez ici
Z = np.zeros((2,4))
for i in range(4):
    Z[:,i] = A@X[:,i]
Z = Z + b
print(Z)
# Vous devez trouver
#[[ 54.  62.  72.  81.]
# [ 86.  97. 116. 131.]]

[[ 54.  62.  72.  81.]
 [ 86.  97. 116. 131.]]


# Implémentation de la couche Dense
Une couche "dense" est une couche qui prend $x$ un vecteur de taille $p$ et rend $y$ un vecteur de taille $q$ tel que
$$y=Ax+b,$$
Où $A$ est une matrice et $b$ un vecteur de taille $q$. La matrice $A$ et le vecteur $b$ sont des paramètres de la couche. 
Nous allons dans un premier temps s'intéresser uniquement à la fonction `__init__`.

La fonction `__init__` prend en argument deux entiers `nb_entree` et `nb_sortie` (notés $p$ et $q$ ici) correspondant à respectivement à la taille des vecteurs d'entrée et la taille des vecteurs de sortie. La fonction `__init__` va initialiser les paramètres $A$ et $b$, ce sont des objets du type `Parameter`. On stocke $A$ en premier, attention, il y a une astuce sur la taille de $b$ qui est claire si vous avez compris la dernière section.

In [29]:
# TEST de la classe
np.random.seed(10)
import Neural as Neur
L=Neur.Dense(3,2)
print('nb_params=',len(L.list_params))
print('structure des params :', [p.data.shape for p in L.list_params])
A,b=L.list_params
print('A=',A.data)
print('b=',b.data)
# Vous devez trouver
#nb_params= 2
#structure des params : [(2, 3), (2, 1)]
#A= [[ 0.49671415 -0.1382643   0.64768854]
# [ 1.52302986 -0.23415337 -0.23413696]]
#b= [[ 0.49671415]
# [-0.1382643 ]]


nb_params= 2
structure des params : [(2, 3), (2, 1)]
A= [[ 0.49671415 -0.1382643   0.64768854]
 [ 1.52302986 -0.23415337 -0.23413696]]
b= [[ 0.49671415]
 [-0.1382643 ]]


On remplit maintenant la fonction `forward` de la classe `Layer`. Etant donné une matrice $X$ de taille $(p,n)$ (où $n$ représente le nombre de données), la fonction `forward` calcule $Y$ de taille $(q,n)$ telle que $Y_j=AX_j+b$ pour tout $j$ entre $1$ et $n$ . On utilisera la section précédente pour faire ce calcul. Attention les matrices $A$ et $b$ doivent être récupérées dans `list_params`

In [37]:
np.random.seed(10)
import Neural as Neur
L = Neur.Dense(3,2)
x =np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
print("###########test1##########")
print(L.forward(x))
# Vous devez trouver
#[[ 6.13130364  7.13744203  8.14358042  9.14971881]
# [-1.89323393 -0.83849441  0.21624512  1.27098464]]
L.list_params[0].data=4*L.list_params[0].data
L.list_params[1].data=2*L.list_params[1].data
print("##############test2############")
print(L.forward(x))
# Vous devez trouver
#[[23.53178627 27.55633983 31.58089339 35.60544695]
# [-7.29640712 -3.07744902  1.14150908  5.36046718]]

###########test1##########
[[ 6.13130364  7.13744203  8.14358042  9.14971881]
 [-1.89323393 -0.83849441  0.21624512  1.27098464]]
##############test2############
[[23.53178627 27.55633983 31.58089339 35.60544695]
 [-7.29640712 -3.07744902  1.14150908  5.36046718]]


 C'était simple... Maintenant il faut coder la rétropropagation pour la couche dense. On rappelle que si la couche est de taille $(p,q)$ avec $n$ données alors la fonction `backward` prend en entrée `hat_y` (noté $\hat y$ ici) un vecteur de taille $(q,n)$, calcule le gradient par rapport à $A$ et $b$ (qui sont donc des vecteurs de taille $(q,p)$ et $q$ respectivement et qui sont notés $hat_A$ et $hat_b$) et rend un vecteur `hat_x` de taille $(p,n)$ (noté $\hat x$ )qui sert à rétropropager le gradient aux couches précédentes. Les formules de calcul sont :

$$\hat x = A^T \hat y$$   
$$\hat A = \hat y x^T$$
$$\hat b[i] = \sum_j \hat y[i,j]$$

Ici $\hat A$ et $\hat b$ doivent être stockés dans les attributs `grad` des paramètres correspondant. 

Vous ferez TRES ATTENTION à ce que pour tout `p` dans `list_params` on ait `p.shape==p.grad.shape` !!!

Implémentez la fonction `backward` de `dense` et testez votre code ci-dessous :

In [39]:
np.random.seed(10)
import Neural as Neur
L = Neur.Dense(3,2)
x=np.random.randn(3,4)
hat_y=np.random.randn(2,4)
L.forward(x)
hat_x=L.backward(hat_y)
print('structure des grads :', [p.grad.shape for p in L.list_params])
print('grad_entree=',hat_x)
print('grad_param=',[p.grad for p in L.list_params])
#Vous devez trouver
#structure des grads : [(2, 3), (2, 1)]
#grad_entree= [[-2.23973888 -2.43027688  1.72913928 -0.18777295]
# [ 0.45111146  0.40843997 -0.20314822  0.00941709]
# [-0.90460752 -0.0335147  -0.99916165  0.25639697]]
#grad_param= [array([[-1.81000929, -2.50953501,  0.21492134],
#       [-3.02942152, -3.32839592,  1.86515132]]), array([-2.98578915, -1.08045531])]

structure des grads : [(2, 3), (2,)]
grad_entree= [[-2.23973888 -2.43027688  1.72913928 -0.18777295]
 [ 0.45111146  0.40843997 -0.20314822  0.00941709]
 [-0.90460752 -0.0335147  -0.99916165  0.25639697]]
grad_param= [array([[-1.81000929, -2.50953501,  0.21492134],
       [-3.02942152, -3.32839592,  1.86515132]]), array([-2.98578915, -1.08045531])]


## Construction de la couche de perte L2
Nous allons maintenant construire une classe qui correspond à la couche de perte $L_2$. Cette couche n'a pas de paramètres, elle a des données $D$ stockées et le forward consiste à calculer 
$$y=\frac{1}{2}\Vert x-D\Vert^2.$$
La variable $y$ est un réel et c'est classiquement la dernière couche du réseau de Neurone, la couche qui nous permet de mesurer l'écart entre $x$ et les données $D$.

Implémentez la couche de perte $L_2$ dans une classe nommée `Loss_L2` et testez le code ci-dessous :


In [44]:
np.random.seed(10)
import Neural as Neur
D=np.random.randn(3,2)
x=np.random.randn(3,2)
L = Neur.Loss_L2(D)
print('nb_params=',len(L.list_params))
print('structure des params :', [p.data.shape for p in L.list_params])

print(L.forward(x))
print(L.backward(2.))
#Vous devez trouver
#nb_params= 0
#structure des params : []
#3.8338361400772234
#[[-2.13214984 -1.2134609 ]
# [ 3.09938345 -0.33243272]
# [-0.37661957  3.84624587]]



nb_params= 0
structure des params : []
3.8338361400772234
[[-2.13214984 -1.2134609 ]
 [ 3.09938345 -0.33243272]
 [-0.37661957  3.84624587]]


# Construction du réseau de Neurones
Nous allons maintenant nous intéresser à la construction du réseau de Neurone en lui même. Un réseau de Neurone est essentiellement une liste de couches  que l'on exécute successivement.

La classe d'un réseau de neurone se nommera `Sequential`. Pour des raisons pratiques, la classe `Sequential` a exactement les mêmes noms d'attributs et de méthode que la classe `Generic`, en fait on pourrait (et c'est même quelquefois très intéressant) considérer les réseaux de neurones comme une couche et faire plusieurs couches de réseau de neurone.

On s'intéresse d'abord à la fonction `__init__`. La fonction `__init__` prend en argument la variable `list_layers` qui est une liste de couches. Cette liste doit être sauvée dans l'attribut `self.list_layers`.

L'attribut `self.list_params` doit être calculé. Par définition, `self.list_params` doit être une liste contenant tous les parmètres des couches de `list_layers`. Il faut donc parcourir la liste des couches et au fur et à mesure ajouter les paramètres des couches à la liste de paramètres du réseau de neurone.

In [ ]:
np.random.seed(10)
import Neural as Neur
D=np.random.randn(4,10)
X=np.random.randn(3,10)

L1=Neur.Dense(3,2)
L2=Neur.Arctan()
L3=Neur.Dense(2,6)
L4=Neur.Arctan()
L5=Neur.Dense(6,4)
L6=Neur.Loss_L2(D)


N=Neur.Sequential([L1,L2,L3,L4,L5,L6])
           
print('nb_params=',len(N.list_params))
print('structure des params :', [p.data.shape for p in N.list_params])
print('params :', [np.linalg.norm(p.data) for p in N.list_params])


# Vous devez obtenir
#nb_params= 6
#structure des params : [(2, 3), (2, 1), (6, 2), (6, 1), (4, 6), (4, 1)]
#params : [1.7648258469835016, 0.5155986489315331, 2.6728107324820174, 1.7648258469835016, 4.725660744989407, 1.7334827235012424]


On va programmer le `forward` et le `backward` du réseau de Neurone. 
Pour le `forward`, il s'agit juste de prendre la variable `x`, de la copier dans une variable `z` et de faire passer cette variable `z` dans chaque couche de la liste. On rend le résultat final. Il est important de copier la variable `x` dans une variable temporaire, sinon notre variable `z` sera modifiée par l'algorithme !!
Pour le backward, il faut juste faire passer le gradient dans la liste à l'envers. Pour parcourir la liste à l'envers, il faut utiliser la fonction `reverse` de python.

In [ ]:
np.random.seed(10)
import Neural as Neur
D=np.random.randn(4,10)
X=np.random.randn(3,10)

L1=Neur.Dense(3,2)
L2=Neur.Arctan()
L3=Neur.Dense(2,6)
L4=Neur.Arctan()
L5=Neur.Dense(6,4)
L6=Neur.Loss_L2(D)
N=Neur.Sequential([L1,L2,L3,L4,L5,L6])

a=N.forward(X)
print('a=',a)
b=N.backward(2.)
print('params :',[np.linalg.norm(p.data) for p in N.list_params])
print('grads :' ,[np.linalg.norm(p.grad) for p in N.list_params])
print('b=',b)
#a= 97.01801355669247
#params : [1.7648258469835016, 0.5155986489315331, 2.6728107324820174, 1.7648258469835016, 4.725660744989407, 1.7334827235012424]
#grads : [26.09283532606045, 37.33604224703243, 52.067713137338835, 107.5237167124915, 104.49242693910662, 71.85707943841926]
#b= [[  1.81182256 -19.05318309   1.90926957   2.51245022   1.75029792
#    4.75127957   1.19170996  20.62802901   6.95648392   0.69150173]
# [ -0.47614424   3.17748395  -0.63867008  -0.46536848  -0.51391082
#   -1.11027516  -0.50349429  -3.56121527  -1.26727391  -0.50345374]
# [  2.03276096   0.02563232   3.74365544   0.53901638   2.59463212
#    3.71228905   3.56320716   1.38874543   1.24397352   4.53919037]]


Et voilà c'est fini... Vous êtes fin prêts à faire des réseaux de neurone maintenant.